In [1]:
import pandas as pd
from pdf2img import *
import os
import fitz

In [3]:
DIR  = "./dataset/Material por ID/"
FILTERED_DIR = "./dataset/filtered/" 

In [2]:
def create_dataset(file : str) -> pd.DataFrame:
    """
    Opens a file that contains the raw dataset, performs data cleaning and returns a cleaned version.
    """
    
    df = pd.read_csv(file)
    
    df.drop(labels=["Guia/Refe","Catedra","Carrera/Facultad","Resuelto","Año"], axis = 1, inplace= True)
    df.rename(columns={"Link":"Nombre"}, inplace= True)
    df.rename(columns={"Unnamed: 13":"Link"}, inplace= True)
    df.dropna(subset="Link", inplace=True)
    df.dropna(subset="Materia", inplace=True)

    indices = df[df["Materia"].str.contains("quimica", case=False)].index
    df[df["Materia"].str.contains("arquitectura", case=False)].index
    df.drop(index= indices, inplace= True)
    indices = df[df["Materia"].str.contains("arquitectura", case=False)].index
    df.drop(index= indices, inplace= True)
    return df

In [3]:
df = create_dataset('./Copia de Base de Material - Rucon - Base de datos (1).csv')


In [4]:
data_subset = df[df['Nombre'].isin(os.listdir("./dataset/Material por ID/"))]

# saving the filtered files

In [8]:
import shutil

In [9]:
for file_name in data_subset['Nombre']:
    file_path = os.path.join("./dataset/Material por ID/", file_name)
    if os.path.isfile(file_path):
        shutil.copy(file_path, "./dataset/filtered/")

# labeling

In [ ]:
import os
import pandas as pd
import fitz
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor

In [6]:


# Paths and constants
OUTPUT_DIR = "dataset/images/"


# Function to process each PDF file
def process_pdf(file_name):
    if file_name.endswith('.pdf'):
        pdf_path = os.path.join(FILTERED_DIR, file_name)
        dir_name = get_file_name(pdf_path)

        # Convert PDF to images and extract labels
        #pdf2_png(pdf_path, OUTPUT_DIR, dir_name)
        labels = get_labels(df, f"{dir_name}.pdf")

        return {
            'ImageName': dir_name,
            'Level': labels[0],
            'University': labels[1],
            'Math Subject': labels[2],
            'Course': labels[3],
            'Type': labels[4],
            'Annotation': labels[5]
        }

# Process PDFs concurrently
image_labels = []
with ThreadPoolExecutor() as executor:
    results = executor.map(process_pdf, os.listdir(FILTERED_DIR))
    for result in results:
        if result:
            image_labels.append(result)

# Convert the list of dictionaries to a DataFrame
labels_df = pd.DataFrame(image_labels)

# Save the DataFrame to a CSV file
labels_df.to_csv("./dataset/labels.csv", index=False)


In [5]:
if __name__ == "__main__":
    with ProcessPoolExecutor() as executor:
        results = executor.map(process_pdf, os.listdir(FILTERED_DIR), df)
        
        for result in results:
            print(result)
            if result:
                image_labels.append(result)

    # Convert the list of dictionaries to a DataFrame
    labels_df = pd.DataFrame(image_labels)

    # Save the DataFrame to a CSV file
    labels_df.to_csv("./dataset/labels.csv", index=False)


TypeError: string indices must be integers, not 'str'